In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
pd.set_option('future.no_silent_downcasting', True)

#### Getting the data

In [3]:
symbol = input("Enter the symbol you want to perform DCF on: ")

In [4]:
ticker = yf.Ticker(symbol)

In [5]:
selected_case = int(input("""
Choose 3 cases:
Case 1: Optimistic case
Case 2: Base case
Case 3: worst case
Enter your choice (1-3): """))

In [6]:
Balance_sheet = ticker.balance_sheet.transpose()
Income_statement = ticker.income_stmt.transpose()

In [10]:
Income_statement['Other revenue'] = Income_statement['Total Revenue'] - Income_statement['Cost Of Revenue']

#### P&L assumption

In [7]:
def forecaster(models: str, year_on_year: bool) -> pd.DataFrame:

    actual = pd.DataFrame(columns=sorted(list(Income_statement.index.year)))
    forecast = pd.DataFrame(columns=[x + max(list(Income_statement.index.year)) for x in range(1,6)]).transpose()

    if models == 'Revenues':
        revenues = pd.DataFrame(Income_statement['Operating Revenue']).rename({'Operating Revenue' : 'Revenues'},axis=1)
    elif models == 'COGS':
        model = 'Cost Of Revenue'
        revenues = pd.DataFrame(Income_statement['Cost Of Revenue'])
    else:
        revenues = pd.DataFrame(Income_statement[models])
    revenues.index = revenues.index.year
    revenues = revenues.transpose()

    actual_revenues = actual.copy()
    actual_revenues = pd.concat([actual_revenues, revenues])
    actual_revenues = actual_revenues.transpose()

    # forecasts items using year on year growth: Revenues, Other revenues
    if year_on_year:
        actual_revenues['y-o-y growth'] = actual_revenues[models].pct_change()

        if selected_case == 1:
            forecast['y-o-y growth'] = max(actual_revenues['y-o-y growth'])
        elif selected_case == 2:
            forecast['y-o-y growth'] = (actual_revenues['y-o-y growth']).mean()
        elif selected_case == 3:
            forecast['y-o-y growth'] = (actual_revenues['y-o-y growth'].mean()) / 2

        forecasted = forecast.copy()
        forecasted[f'forecast {models}'] = actual_revenues[models].iloc[-1] * (1 + forecasted['y-o-y growth'])    
        
        for i in range(1, len(forecasted)):
            forecasted.loc[forecasted.index.min() + i,f'forecast {models}'] = forecasted.loc[forecasted.index.min() + i -1, f'forecast {models}'] * (1 + forecasted['y-o-y growth'].iloc[0])

        return forecasted   

    # forecasts items using % of revenue: COGS
    else:
        revenue = pd.DataFrame(Income_statement['Operating Revenue']).sort_index()
        revenue.index = revenue.index.year
        actual_revenues['% of revenue'] = actual_revenues[model] / revenue['Operating Revenue']    
        
        if selected_case == 1:
            forecast['% of revenue'] = max(actual_revenues['% of revenue'])
        elif selected_case == 2:
            forecast['% of revenue'] = (actual_revenues['% of revenue']).mean()
        elif selected_case == 3:
            forecast['% of revenue'] = (actual_revenues['% of revenue'].mean()) / 2   

        forecasted = forecast.copy()
        forecast_revenue = forecaster('Revenues', True)
        forecasted[f'forecast {models}'] = forecast_revenue['forecast Revenues'] * forecasted['% of revenue']         
    
        return forecasted     

##### Modeling Revenue from sales and services

In [13]:
forecast_revenue = forecaster('Revenues', True)
forecast_revenue

,y-o-y growth,forecast Revenues
2024,0.070616,2.268797e+11
2025,0.070616,2.429011e+11
2026,0.070616,2.600539e+11
2027,0.070616,2.784180e+11
2028,0.070616,2.980788e+11


##### Modeling Other revenues


In [11]:
forecast_oth_revenue = forecaster('Other revenue',True)
forecast_oth_revenue

,y-o-y growth,forecast Other revenue
2024,0.07378,1.568277e+11
2025,0.07378,1.683985e+11
2026,0.07378,1.808229e+11
2027,0.07378,1.941640e+11
2028,0.07378,2.084894e+11


##### Modeling Cost of goods sold

In [14]:
forecast_Cogs = forecaster('COGS', False)
forecast_Cogs

,% of revenue,forecast COGS
2024,0.157464,3.572545e+10
2025,0.157464,3.824825e+10
2026,0.157464,4.094920e+10
2027,0.157464,4.384089e+10
2028,0.157464,4.693677e+10
